In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import os
import cleaning
import numpy as np

In [3]:
import malaya

In [4]:
from glob import glob

files = glob('../pure-text/dumping-*.txt')
files

['../pure-text/dumping-instagram.txt',
 '../pure-text/dumping-parliament.txt',
 '../pure-text/dumping-twitter.txt',
 '../pure-text/dumping-iium.txt',
 '../pure-text/dumping-wiki.txt',
 '../pure-text/dumping-news.txt',
 '../pure-text/dumping-watpadd.txt',
 '../pure-text/dumping-pdf.txt',
 '../pure-text/dumping-common-crawl.txt']

In [5]:
with open('../pure-text/dumping-news.txt') as fopen:
    data = fopen.read().split('\n')
    
data[:10]

['Lawatan Putera Mahkota Arab Saudi bincang pelaburan, ekonomi',
 'KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.',
 'Menteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.',
 '"Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).',
 '"Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.',
 '"Ia banyak berkisar ekonomi dan perniagaan," katanya kepada pemberita selepas hadir di majlis meja bulat bersama belia di Institut Diplomasi dan Hubungan Luar Negeri, di sini, hari ini.',
 'Lawatan Putera Mahkota Arab Saudi ke Malaysia akan berlangsung selama dua hari berakhir Isnin ini.',
 '',
 'I

In [6]:
results, result = [], []
for i in data:
    if not len(i) and len(result):
        results.append(' '.join(result))
        result = []
    else:
        result.append(i)

In [7]:
from tqdm import tqdm

selected = []
for r in tqdm(results):
    if r[:30].isupper():
        continue
    if len(r) < 200:
        continue
    if 'ISBN' in r or 'cipta terpelihara' in r or 'rights reserved' in r:
        continue
    selected.extend(malaya.text.function.split_into_sentences(r, minimum_length = 20))

100%|██████████| 105469/105469 [01:38<00:00, 1073.27it/s]


In [8]:
selected[:100]

['Lawatan Putera Mahkota Arab Saudi bincang pelaburan, ekonomi KUALA LUMPUR: Lawatan Putera Mahkota Arab Saudi, Mohammed bin Salman ke Malaysia pada Ahad ini terarah kepada pertemuan bagi membincangkan isu ekonomi dan pelaburan antara kedua-dua negara.',
 'Menteri Luar, Datuk Saifuddin Abdullah, berkata tujuan lawatan kerabat diraja Arab Saudi itu akan diperjelaskan oleh Kementerian Hal Ehwal Ekonomi.',
 '"Saya rasa elok perkara itu (lawatan Putera Mahkota Arab Saudi) dijelaskan sendiri oleh (Menteri Hal Ehwal Ekonomi) Datuk Seri Mohamed Azmin (Ali).',
 '"Selain saya (keluarkan kenyataan mengenai lawatan itu), kamu (pemberita) juga boleh ajukan soalan kepada Datuk Seri Mohamed Azmin.',
 '"Ia banyak berkisar ekonomi dan perniagaan," katanya kepada pemberita selepas hadir di majlis meja bulat bersama belia di Institut Diplomasi dan Hubungan Luar Negeri, di sini, hari ini.',
 'Lawatan Putera Mahkota Arab Saudi ke Malaysia akan berlangsung selama dua hari berakhir Isnin ini.',
 'Isu halal 

In [9]:
import tensorflow_text  
import tensorflow as tf

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], '../t5/output/1598704247')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../t5/output/1598704247/variables/variables


In [10]:
from tqdm import tqdm
import re

def cleaning(string):
    string = re.sub('[^\'"A-Za-z\- ]+', ' ', string)
    string = re.sub(r'[ ]+', ' ', string.lower()).strip()
    return string

batch_size = 20
output = []

for i in tqdm(range(0, len(selected) // batch_size * batch_size, batch_size)):
    batch = selected[i: i + batch_size]
    batch_x = [f'parafrasa: {s}' for s in batch]
    r = pred(batch_x)
    
    for k in range(batch_size):
        if cleaning(batch[k]) == cleaning(r[k].decode()):
            continue
        output.append((batch[k], r[k].decode()))

  9%|▉         | 7306/82124 [7:17:52<75:52:33,  3.65s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 45%|████▌     | 37245/82124 [36:36:02<40:52:21,  3.28s/it] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 62%|██████▏   | 51245/82124 [50:22:43<30:21:25,  3.54s/it] 


KeyboardInterrupt: 

In [11]:
len(output)

1009479

In [13]:
import json
with open('semisupervised-news-paraphrase.json', 'w') as fopen:
    json.dump(output, fopen)